In [20]:
import sys
try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages') 
except: 
    pass
import cv2 
import numpy as np 
import matplotlib.pyplot as plt



cap = cv2.VideoCapture("/home/kamakshi/Documents/perception/project2/DataSet/challenge_video.mp4")
K = np.array([[ 1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [ 0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [ 0.00000000e+00,0.00000000e+00,1.00000000e+00]]) 
dist = np.array([ -2.42565104e-01,-4.77893070e-02, -1.31388084e-03, -8.79107779e-05, 2.20573263e-02]) 
while(True): 
        ret,frame = cap.read() 
        frame = cv2.undistort(frame,K,dist) 


        hls = cv2.cvtColor(frame, cv2.COLOR_RGB2HLS)
        h_channel = hls[:,:,0]
        
        s_channel = hls[:,:,2]
        l_channel = hls[:,:,1]
        #cv2.imshow("s",l_channel)
        # Grayscale image
        # NOTE: we already saw that standard grayscaling lost color information for the lane lines
        # Explore gradients in other colors spaces / color channels to see what might work better
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
 
        # Sobel x
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
        abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
        scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
 
        # Threshold x gradient
        thresh_min = 20
        thresh_max = 100
        sxbinary = np.zeros([scaled_sobel.shape[0],scaled_sobel.shape[1]])
        sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
 
        # Threshold color channel
        s_thresh_min = 170
        s_thresh_max = 255
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
 
        # Combine the two binary thresholds
        combined_binary = np.zeros_like(sxbinary)
        combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
        
      

        #cv2.imshow('binary',combined_binary)
        
        shape = combined_binary.shape[::-1] # (width,height)
        w = shape[0]
        h = shape[1]
        transform_src = np.float32([ [w/2-50,h/2+110],[w/2+50,h/2+110],[w/2+200,h/2+250],[w/2-200,h/2+250]])
        transform_dst = np.float32([ [300,300],[500,300],[500,500],[300,500]])
        
        
        
        M = cv2.getPerspectiveTransform(transform_src, transform_dst)
        
        warped_image = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]), flags=cv2.INTER_NEAREST)
        warped_image = np.uint8(255*warped_image/np.max(warped_image))
        warped_image = cv2.bilateralFilter(warped_image,9,75,75)
        warped_image= cv2.Canny(warped_image,75,255)
        #cv2.imshow("warped",warped_image)
        histogram = np.sum(warped_image[warped_image.shape[0]/2:,:], axis=0)
        
        #Create an output image to draw on and  visualize the result
        out_img = np.dstack((warped_image, warped_image, warped_image))*255
#         print(out_img)
        
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        nwindows = 9
        window_height = np.int(warped_image.shape[0]/nwindows)
        nonzero = warped_image.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        #current window getting updated
        leftx_current = leftx_base
        rightx_current = rightx_base
        margin = 100
        minpix = 50
        
        
        #right and left lane pixel
        left_lane_inds = []
        right_lane_inds = []
        
        
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low =warped_image.shape[0] - (window+1)*window_height
            win_y_high = warped_image.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            
            
            
            
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
                
                
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
#         print("right",right_lane_inds.shape)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 
        
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        
        # Generate x and y values for plotting
        ploty = np.linspace(0, warped_image.shape[0]-1, warped_image.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = warped_image.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        out_img = np.dstack((warped_image,warped_image, warped_image))*255
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        cv2.imshow("lane",out_img)
       
        
        
#         ym_per_pix = 30/720 # meters per pixel in y dimension
#         xm_per_pix = 3.7/700 # meters per pixel in x dimension
 
#         y_eval = np.max(ploty)
 
#         fit_cr_left = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2)
#         curverad_left = ((1 + (2 * left_fit[0] * y_eval / 2. + fit_cr_left[1]) ** 2) ** 1.5) / np.absolute(2 * fit_cr_left[0])
#         fit_cr_right = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)
#         curverad_right = ((1 + (2 * left_fit[0] * y_eval / 2. + fit_cr_right[1]) ** 2) ** 1.5) / np.absolute(2 * fit_cr_right[0])
        
        
        if cv2.waitKey(1)& 0xff==ord('q'):
            cv2.destroyAllWindows() 
            break

In [21]:

try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt

cap = cv2.VideoCapture("/home/kamakshi/Documents/perception/project2/DataSet/project_video.mp4")
K = np.array([[  1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [  0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist = np.array([ -2.42565104e-01,-4.77893070e-02,  -1.31388084e-03,  -8.79107779e-05,
    2.20573263e-02])
while(True):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    undist = cv2.undistort(gray,K,dist)
    
    h = gray.shape[0]
    w = gray.shape[1]
    
    src = np.array([[w/2-50,h/2+110],[w/2+50,h/2+110],[w/2+200,h/2+250],[w/2-200,h/2+250]])
    dst = np.array([[300,300],[500,300],[500,500],[300,500]])
    H,flag = cv2.findHomography(src,dst)
    out= cv2.warpPerspective(undist,H,(w,h-200))
    blur = cv2.bilateralFilter(out,9,75,75)
    median = cv2.medianBlur(blur,5)
    
    gray = cv2.cvtColor(median,cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(gray,cv2.CV_64F,1,0)
    abs_sobel = np.absolute(sobel)
    scaled = np.uint8(255*abs_sobel/np.max(abs_sobel))

    
    mask_x = cv2.inRange(abs_sobel,20,100)
    seg_x = cv2.bitwise_and(scaled,scaled,mask=mask_x)
  
    
#     mask1 = cv2.inRange(median,np.array([20,50,50]),np.array([30,255,255]))
    mask = cv2.inRange(median,np.array([0,0,170]),np.array([200,255,255]))
    seg = cv2.bitwise_and(median,median,mask=mask)
    img = cv2.bitwise_or(seg,seg,mask=mask_x)
#     scaled = np.uint8(255*erosion/np.max(erosion))
    
#     blur = cv2.bilateralFilter(binary,9,75,75)
#     canny = cv2.Canny(blur,75,255)
#     plt.imshow(sobel,cmap="binary")
#     sobel = cv2.Sobel(sobel,cv2.CV_64F,1,0)


    histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
    histo = np.dstack((img, img, img))*255
   
     #peaks of the histogram  
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
   
    
    #sliding window
    
    
    nwindows = 9
    window_height = np.int(img.shape[0]/nwindows)  
    print(window_height)
    nonzero = img.nonzero()
    print(nonzero)
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    
    leftx_current = leftx_base   #peaks for left and right for this frame
    rightx_current = rightx_base
    
    margin = 100
    minpix = 50
    
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        
        
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
       
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        
        
        wleft = cv2.rectangle(img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        wright = cv2.rectangle(img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]     
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        print(right_lane_inds)
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
#             print(leftx_current)
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
#             print(rightx_current)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
                                    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    out_img = np.dstack((img, img, img))*255
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
                                


        
    cv2.imshow('saturation',)
    cv2.imshow('segmant',seg_x)
    cv2.imshow('canny',canny)
    cv2.imshow('mask',mask)
    if cv2.waitKey(1)& 0xff==ord('q'):
        cv2.destroyAllWindows()
        break

57
(array([  0,   0,   0, ..., 519, 519, 519]), array([131, 131, 131, ..., 287, 287, 287]), array([0, 1, 2, ..., 0, 1, 2]))
[array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
[arr

TypeError: expected non-empty vector for x